In [3]:
import os
import rasterio as rio
import numpy as np
import geopandas as gpd
import pandas as pd
from rasterio.warp import calculate_default_transform, transform, reproject, Resampling


In [6]:
import ee
import geemap
import geopandas as gpd
import os

ee.Initialize()
ee.Authenticate()

Enter verification code:  4/1AfJohXkPu-lLOcv8IsZTFNbwrOlz3akT20PmVWx9sXsgftWaaw1xcSUn3RQ



Successfully saved authorization token.


In [ ]:
## MAPBIOMAS download

years=[2019] ##, 2020, 2021, 2022
out_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/MB_PRY"
grid_file="/raida-cel/r/downspout-cel/paraguay_lc/Segmentations/PY_grid_8858.gpkg"

gdf = gpd.read_file(grid_file)
gdf = gdf.set_crs(gdf.crs)
gdf_web = gdf.to_crs('EPSG:4326')
#gdf_web =  gdf_web[gdf_web['UNQ'].isin(gridCells_TODO)] # subset UNQ rows that are in gridCells_TODO list 

band_list = ['classification_'+str(yr) for yr in years] 
for k, cell in gdf_web.iterrows():
    aoi = ee.Geometry.Rectangle([cell.geometry.bounds[0], cell.geometry.bounds[1], cell.geometry.bounds[2], cell.geometry.bounds[3]])
    UNQ = int(cell['UNQ'])

    dst = ee.Image( "projects/mapbiomas-public/assets/paraguay/collection1/mapbiomas_paraguay_collection1_integration_v1")
    bands = dst.select(band_list)
    projection = bands.projection().getInfo() ## arbitrary band, to grab CRS extent info
    
    for band in band_list: # save one band at a time 
        out_name=os.path.join(out_dir, "UNQ"+str(UNQ)+"."+band+".tif")
        if not os.path.exists(out_name):
            # full_out_dir = os.path.join(out_dir, band.split("_")[-1])
            # if not os.path.exists(full_out_dir):
            #     os.makedirs(full_out_dir)
            geemap.ee_export_image(bands.select(band), 
                                filename=out_name, 
                                crs=projection.get('crs'), crs_transform=projection.get('transform'),
                                region=aoi, file_per_band=True)

In [ ]:
start_date = '2017-01-01'
end_date = '2018-01-01'

region = ee.Geometry.BBox(8.1, 13.1,  8.7, 13.9)
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(region)

dw_class = geemap.dynamic_world(region, start_date, end_date, return_type='class')
dw_vis = {"min": 0, "max": 8, "palette": [ "#419BDF", "#88B053", "#7A87C6", "#E49635", "#DFC35A", "#C4281B", "#A59B8F", "#B39FE1", ],}
Map.addLayer(dw_class, dw_vis, 'DW Land Cover', True)
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')

buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons") ## v3: inference carried out during May 2023
buildings_AOI = buildings.filterBounds(region)
Map.addLayer(buildings_AOI)

Map

In [129]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio

from sklearn.metrics import confusion_matrix

In [146]:
samp_pts="/home/l_sharwood/code/fishnet_1km_label.shp"
##samp_pts="/home/l_sharwood/code/samp_500m.shp"

rast_list=["/home/l_sharwood/code/Baumann_reclass_2022_8858.tif",
##"/home/l_sharwood/code/MB_PRY_reclass_2022_8858.tif",
"/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/MB_PRY/reclass_2/MB_PRY_reclass_2022_8858.tif", 
##"/home/l_sharwood/code/CEL_reclass_2022_8858_10m.tif",
"/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/CEL/reclass_2/CEL_reclass_2022_8858.tif"
]


field_pts = gpd.read_file(samp_pts)
coords_list = list(zip([i.x for i in field_pts.geometry.to_list()],  [i.y for i in field_pts.geometry.to_list()]))
field_pts=field_pts[["geometry"]]

for rast in rast_list:
    prod_name=os.path.basename(rast).replace("_reclass_2022_8858", "")[:-4].replace("_10m", "")
    with rio.open(rast) as src:
        ## create column for raster, row is raster value at each coordinate
        field_pts[prod_name] = [x[0] for x in src.sample(coords_list)]
## remove rows where they're ALL zero
df = field_pts.loc[field_pts['CEL'] * field_pts['Baumann'] * field_pts['MB_PRY'] != 0]

df.to_csv(os.path.basename(samp_pts).replace(".shp", "_2.csv"))

In [147]:
true_pred = ["CEL", "MB_PRY"]

## subset cols 
matrix_df = df[[true_pred[0], true_pred[1]]]
## delete if ANY are zero
matrix_df = matrix_df[(matrix_df != 0).all(1)]

## first is true value, second
true = matrix_df[true_pred[0]]
pred =  matrix_df[true_pred[1]]
agree_arr = confusion_matrix(true, pred)
df_ag = pd.DataFrame(agree_arr)
df_ag.to_csv("/home/l_sharwood/code/"+os.path.basename(samp_pts)[:-4]+"_agree_"+true_pred[0]+"_"+true_pred[1]+"_2.csv")

In [148]:
true_pred = ["CEL", "Baumann"]

## subset cols 
matrix_df = df[[true_pred[0], true_pred[1]]]
## delete if ANY are zero
matrix_df = matrix_df[(matrix_df != 0).all(1)]

## first is true value, second
true = matrix_df[true_pred[0]]
pred =  matrix_df[true_pred[1]]
agree_arr = confusion_matrix(true, pred)
df_ag = pd.DataFrame(agree_arr)
df_ag.to_csv("/home/l_sharwood/code/"+os.path.basename(samp_pts)[:-4]+"_agree_"+true_pred[0]+"_"+true_pred[1]+"_2.csv")

In [149]:
true_pred = ["MB_PRY", "Baumann"]

## subset cols 
matrix_df = df[[true_pred[0], true_pred[1]]]
## delete if ANY are zero
matrix_df = matrix_df[(matrix_df != 0).all(1)]

## first is true value, second
true = matrix_df[true_pred[0]]
pred =  matrix_df[true_pred[1]]
agree_arr = confusion_matrix(true, pred)
df_ag = pd.DataFrame(agree_arr)
df_ag.to_csv("/home/l_sharwood/code/"+os.path.basename(samp_pts)[:-4]+"_agree_"+true_pred[0]+"_"+true_pred[1]+"_2.csv")

In [ ]:
## ARCHIVE

In [ ]:
### ## reclass -> reproject -> mosaic -> resize to 10m (in Arc) -> TILE

grid_shape="/raida-cel/r/downspout-cel/paraguay_lc/Segmentations/PY_grid_8858.gpkg"
prodBase_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/CEL"
##out_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/Baumann"
out_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/MB_PRY"

####################

grids=sorted([int(i.split("_")[3][-4:]) for i in os.listdir(prodBase_dir) if i.endswith(".tif") and "reclass" in i])
grid_gdf = gpd.read_file(grid_shape)
for UNQ in sorted(grids):
    bounds = grid_gdf[grid_gdf['UNQ'] == UNQ].geometry.bounds.iloc[0]
    boundary = (float(bounds[0]), float(bounds[2]), float(bounds[1] ), float(bounds[3]))
    for pred_rast in [os.path.join(out_dir, i) for i in os.listdir(out_dir) if "reclass" in i and i.endswith("8858_10m.tif")] : 
        with rio.open(pred_rast) as srcA:
            gt = srcA.transform
            offset = img_to_bbox_offsets(gt, boundary)
            new_gt = rio.Affine(gt[0], gt[1], (gt[2] + (offset[0] * gt[0])), 0.0, gt[4], (gt[5] + (offset[1] * gt[4])))
            A_arr = srcA.read(window=Window(offset[0], offset[1], offset[2], offset[3]))   
            copy_meta=srcA.meta.copy()
            
        copy_meta.update({"height":A_arr.shape[1], "width":A_arr.shape[2], "transform":new_gt})
        out_name = os.path.join(out_dir, os.path.basename(out_dir)+"_2022_8858_10m_"+"UNQ"+str(UNQ)+".tif")
        with rio.open(out_name, 'w', **copy_meta) as dst:
            dst.write(A_arr)
        print(out_name)

In [ ]:
## COMPARE UNQ TILES

prodB_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/Baumann"
##prodB_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/MB_PRY"

###################

prodBase_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/CEL"
out_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/CEL_v_"+os.path.basename(prodB_dir)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
grid_gdf = gpd.read_file(grid_shape)
grids=sorted([int(i.split("_")[3][-4:]) for i in os.listdir(prodBase_dir) if i.endswith(".tif") and "reclass" in i])
for UNQ in sorted(grids):
    
    bounds = grid_gdf[grid_gdf['UNQ'] == UNQ].geometry.bounds.iloc[0]
    boundary = (float(bounds[0]), float(bounds[2]), float(bounds[1] ), float(bounds[3]))
    
    base_rast = os.path.join(prodBase_dir, os.path.basename(prodBase_dir)+"_reclass_2022_"+"UNQ"+str(UNQ)+"_8858.tif")
    with rio.open(base_rast) as src:
        gt = src.transform
        offset = img_to_bbox_offsets(gt, boundary)  
        new_gt = rio.Affine(gt[0], gt[1], (gt[2] + (offset[0] * gt[0])), 0.0, gt[4], (gt[5] + (offset[1] * gt[4])))
        CEL_arr = src.read(window=Window(offset[0], offset[1], offset[2], offset[3]))   

    other_prod = os.path.join(prodB_dir, os.path.basename(prodB_dir)+"_2022_8858_10m_"+"UNQ"+str(UNQ)+".tif")
    if os.path.exists(other_prod):
        with rio.open(other_prod) as src:
            gt = src.transform
            offset = img_to_bbox_offsets(gt, boundary)  
            new_gt = rio.Affine(gt[0], gt[1], (gt[2] + (offset[0] * gt[0])), 0.0, gt[4], (gt[5] + (offset[1] * gt[4])))
            other_arr = src.read(window=Window(offset[0], offset[1], offset[2], offset[3]))   
            copy_meta = src.meta.copy()
        ## if the other array has any values (isn't a blank tile)
        if other_arr.max() > 0:
            if CEL_arr.shape==other_arr.shape:
                boolarr=(CEL_arr==other_arr)
                print(str(np.int32(np.count_nonzero(boolarr)/(other_arr.shape[2]*other_arr.shape[1])*100))+'% of pixels are the same in grid cell '+str(UNQ))

                diff_arr = A_arr.copy()
                diff_arr=np.where(diff_arr > 0, 99, 99) ## start w/ array of 99's (don't use 0-nodata is 0)
                diff_arr=np.where(((CEL_arr==0) | (other_arr==0)), 0, diff_arr) ## where either raster is 0(no data), make the comparison raster 0
                diff_arr=np.where(((CEL_arr==1) & (other_arr==1)), 1, diff_arr) ## 1 where they're both crop
                diff_arr=np.where(((CEL_arr==2) & (other_arr==2)), 2, diff_arr) ## 2 where they're both grass
                diff_arr=np.where(((CEL_arr==3) & (other_arr==3)), 3, diff_arr) ## 3 where they're both forest
                diff_arr=np.where(((CEL_arr==4) & (other_arr==4)), 4, diff_arr) ## 4 where they're both other
                diff_arr=np.where(((CEL_arr==1) & (other_arr==2)), 12, diff_arr) ## 12 where A says crop but B says grass
                diff_arr=np.where(((CEL_arr==2) & (other_arr==1)), 21, diff_arr) ## 21 where A says grass but B says crop
                diff_arr=np.where(((CEL_arr==1) & (other_arr==3)), 13, diff_arr) ## 13 where A says crop but B says forest
                diff_arr=np.where(((CEL_arr==3) & (other_arr==1)), 31, diff_arr) ## 31 where A says forest but B says crop
                diff_arr=np.where(((CEL_arr==1) & (other_arr==4)), 14, diff_arr) ## 14 where A says crop but B says other
                diff_arr=np.where(((CEL_arr==4) & (other_arr==1)), 41, diff_arr) ## 41 where A says other but B says crop

                ## not considering grass misclassed as forest or other 
                copy_meta.update({"height":diff_arr.shape[1], "width":diff_arr.shape[2], "transform":new_gt})
                out_name = os.path.join(out_dir, os.path.basename(out_dir)+"_UNQ"+str(UNQ)+".tif")

                with rio.open(out_name, 'w', **copy_meta) as dst:
                    dst.write(diff_arr)

In [ ]:
## value counts
out_dir="/raida-cel/r/downspout-cel/paraguay_lc/lc_prods/CEL_v_Baumann"

compar_rasts = sorted([os.path.join(out_dir,i) for i in os.listdir(out_dir) if i.endswith(".tif")])
for raster in compar_rasts:
    with rio.open(raster) as src:
        arr=src.read(1)
        value_counts = np.unique(arr, return_counts=True)
        print(dict(zip(value_counts[0], value_counts[1])))
        classed_pix_count = np.count_nonzero(arr)
        